In [38]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import PIL


In [39]:
# prepare data_dir
data_dir = "data_dir"
train_dir = pathlib.Path(data_dir + "/train")
test_dir = pathlib.Path(data_dir + "/test")


In [51]:
# get train and test images
train_images_paths = list(train_dir.glob("*/*.*"))
test_images_paths = list(test_dir.glob("*/*.*"))

train_images = np.array([])
test_images = np.array([])

i = 1;
for fname in train_images_paths:
    print(i, "/", len(train_images_paths))
    train_images = np.append(train_images, PIL.Image.open(fname))
    i += 1

i = 1;
for fname in test_images_paths:
    print(i, "/", len(train_images_paths))
    test_images = np.append(test_images, PIL.Image.open(fname))
    i += 1

print(train_images)
print(test_images)


1 / 244
2 / 244
3 / 244
4 / 244
5 / 244
6 / 244
7 / 244
8 / 244
9 / 244
10 / 244
11 / 244
12 / 244
13 / 244
14 / 244
15 / 244
16 / 244
17 / 244
18 / 244
19 / 244
20 / 244
21 / 244
22 / 244
23 / 244
24 / 244
25 / 244
26 / 244
27 / 244
28 / 244
29 / 244
30 / 244
31 / 244
32 / 244
33 / 244
34 / 244
35 / 244
36 / 244
37 / 244
38 / 244
39 / 244
40 / 244
41 / 244
42 / 244
43 / 244
44 / 244
45 / 244
46 / 244
47 / 244
48 / 244
49 / 244
50 / 244
51 / 244
52 / 244
53 / 244
54 / 244
55 / 244
56 / 244
57 / 244
58 / 244
59 / 244
60 / 244
61 / 244
62 / 244
63 / 244
64 / 244
65 / 244
66 / 244
67 / 244
68 / 244
69 / 244
70 / 244
71 / 244
72 / 244
73 / 244
74 / 244
75 / 244
76 / 244
77 / 244
78 / 244
79 / 244
80 / 244
81 / 244
82 / 244
83 / 244
84 / 244
85 / 244
86 / 244
87 / 244
88 / 244
89 / 244
90 / 244
91 / 244
92 / 244
93 / 244
94 / 244
95 / 244
96 / 244
97 / 244
98 / 244
99 / 244
100 / 244
101 / 244
102 / 244
103 / 244
104 / 244
105 / 244
106 / 244
107 / 244
108 / 244
109 / 244
110 / 244
111 / 24

In [ ]:
# train and test .npy path 
train_npy_path = train_dir + "train.npy"
test_npy_path = test_dir + "test.npy"

In [ ]:
# save train and test images
np.save(train_npy_path, train_images)
np.save(test_npy_path, test_images)

In [ ]:
# load train and test images 
train_images = np.load(train_npy_path)
test_images = np.load(test_npy_path)

In [ ]:
# set parameters for the loader
img_height = 1376
img_width = 1038
batch_size = 3


In [ ]:
# prepare training data set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)


In [ ]:
# prepare validation data set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)


In [ ]:
# prepare class names
class_names = train_ds.class_names


In [ ]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
# normalize data from [0, 255] to [0, 1]
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)


In [ ]:
# prepare normalized data set and use it
train_normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
train_image_batch, train_labels_batch = next(iter(train_normalized_ds))

val_normalized_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
val_image_batch, val_labels_batch = next(iter(val_normalized_ds))


In [ ]:
# create model
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(
            1./255, input_shape=(img_height, img_width, batch_size)),
    tf.keras.layers.Flatten(input_shape=(img_height, img_width)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names))
])


In [ ]:
# compile model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
# model summary
model.summary()


In [ ]:
# train model
epochs = 3
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


In [ ]:
# Evaluate accuracy
test_loss, test_acc = model.evaluate(val_image_batch.numpy(),  class_names, verbose=2)

print('\nTest accuracy:', test_acc)
